# Volumetric Imaging Example

This notebook demonstrates how to use the ndx-microscopy extension for storing and analyzing volumetric imaging data in NWB format. Volumetric imaging differs from planar imaging in several key aspects:

- Acquires data in three dimensions (x, y, z) rather than just two (x, y)
- Uses specialized classes for handling volumetric data (VolumetricImagingSpace, VolumetricMicroscopySeries)
- Typically has lower temporal resolution due to the time needed to scan through multiple z-planes
- Requires 3D ROI segmentation and analysis

## Import Required Libraries

Import necessary packages from NWB, ndx-microscopy, and ndx-ophys-devices extensions, along with other utility libraries.

In [ ]:
from datetime import datetime
from uuid import uuid4
import numpy as np
from pynwb import NWBFile, NWBHDF5IO

from ndx_microscopy import (
    MicroscopeModel,
    Microscope,
    MicroscopyRig,
    MicroscopyChannel,
    LineScan,
    VolumetricImagingSpace,
    VolumetricMicroscopySeries,
    VolumetricSegmentation,
    SummaryImage,
    MicroscopyResponseSeries,
    MicroscopyResponseSeriesContainer
)

from ndx_ophys_devices import (
    ExcitationSourceModel,
    PulsedExcitationSource,
    BandOpticalFilterModel,
    BandOpticalFilter,
    DichroicMirrorModel,
    DichroicMirror,
    PhotodetectorModel,
    Photodetector,
    Indicator
)

## Create NWB File

Initialize a new NWB file with basic session information for volumetric imaging.

In [ ]:
nwbfile = NWBFile(
    session_description='Volumetric imaging session',
    identifier=str(uuid4()),
    session_start_time=datetime.now(),
    lab='Neural Dynamics Lab',
    institution='University of Neuroscience',
    experiment_description='Volumetric imaging in cortex'
)

## Set Up Microscope and Components

Configure the microscope and its various components for volumetric imaging. The setup is similar to planar imaging, but optimized for volume acquisition:
- Microscope model and instance
- Laser source for deep tissue penetration
- Optical components (filters and dichroic)
- PMT detector
- GCaMP6f indicator

In [ ]:
# Set up microscope model
microscope_model = MicroscopeModel(
    name='volume-model',
    description='Volumetric imaging microscope model',
    model_number='volume-001',
    manufacturer='Custom Build'
)
nwbfile.add_device(microscope_model)

# Set up microscope with technique
microscope = Microscope(
    name='volume-scope',
    description='Custom volumetric imaging microscope',
    serial_number='volume-serial-001',
    model=microscope_model,
    technique='acousto-optical deflectors'
)
nwbfile.add_device(microscope)

In [ ]:
# Configure laser model and instance
excitation_source_model = ExcitationSourceModel(
    name="chameleon_model",
    manufacturer="Coherent",
    model_number="Chameleon Ultra II",
    description="Excitation source model for volumetric imaging",
    source_type="laser",
    excitation_mode="two-photon",
    wavelength_range_in_nm=[800.0, 1000.0]
)
nwbfile.add_device(excitation_source_model)

laser = PulsedExcitationSource(
    name='laser',
    description='Femtosecond pulsed laser for volumetric imaging',
    serial_number="CU2-SN-123456",
    model=excitation_source_model,
    power_in_W=2.0,
    peak_power_in_W=100000.0,
    peak_pulse_energy_in_J=1.25e-9,
    pulse_rate_in_Hz=80.0e6,
)
nwbfile.add_device(laser)

In [ ]:
# Configure optical components
excitation_filter_model = BandOpticalFilterModel(
    name="excitation_filter_model",
    filter_type="Bandpass",
    manufacturer="Semrock",
    model_number="FF01-920/80",
    center_wavelength_in_nm=920.0,
    bandwidth_in_nm=80.0
)
nwbfile.add_device(excitation_filter_model)

excitation_filter = BandOpticalFilter(
    name='excitation_filter',
    description='Excitation filter for volumetric imaging',
    serial_number="EF-SN-123456",
    model=excitation_filter_model
)
nwbfile.add_device(excitation_filter)

dichroic_mirror_model = DichroicMirrorModel(
    name="primary_dichroic_model",
    manufacturer="Semrock",
    model_number="FF757-Di01",
    cut_on_wavelength_in_nm=757.0,
    cut_off_wavelength_in_nm=750.0,
    transmission_band_in_nm=[757.0, 1100.0],
    reflection_band_in_nm=(400.0, 750.0),
    angle_of_incidence_in_degrees=45.0
)
nwbfile.add_device(dichroic_mirror_model)

dichroic = DichroicMirror(
    name="primary_dichroic",
    description="Dichroic mirror for volumetric imaging",
    serial_number="DM-SN-123456",
    model=dichroic_mirror_model
)
nwbfile.add_device(dichroic)

emission_filter_model = BandOpticalFilterModel(
    name="emission_filter_model",
    filter_type="Bandpass",
    manufacturer="Semrock",
    model_number="FF01-510/84",
    center_wavelength_in_nm=510.0,
    bandwidth_in_nm=84.0
)
nwbfile.add_device(emission_filter_model)

emission_filter = BandOpticalFilter(
    name='emission_filter',
    description='Emission filter for GCaMP6f',
    serial_number="EF-SN-123456",
    model=emission_filter_model
)
nwbfile.add_device(emission_filter)

photodetector_model = PhotodetectorModel(
    name="pmt_model",
    detector_type="PMT",
    manufacturer="Hamamatsu",
    model_number="R6357",
    gain=70.0,
    gain_unit="dB"
)
nwbfile.add_device(photodetector_model)

detector = Photodetector(
    name='pmt',
    description='PMT detector for fluorescence detection',
    serial_number="PMT-SN-123456",
    model=photodetector_model
)
nwbfile.add_device(detector)

In [ ]:
# Create indicator
indicator = Indicator(
    name='gcamp6f',
    label='GCaMP6f',
    description='Calcium indicator for volumetric imaging',
    manufacturer='Addgene',
    injection_brain_region='Visual cortex',
    injection_coordinates_in_mm=[-2.5, 3.2, 0.5]
)

## Configure Microscopy Rig

Set up the microscopy rig, connecting all the previously defined components. The configuration is similar to planar imaging but may require additional considerations for maintaining signal quality throughout the volume.

In [ ]:
# Create microscopy rig
microscopy_rig = MicroscopyRig(
    name='volume_rig',
    description='Volumetric microscopy rig',
    microscope=microscope,
    excitation_source=laser,
    excitation_filter=excitation_filter,
    dichroic_mirror=dichroic,
    photodetector=detector,
    emission_filter=emission_filter
)

# Create microscopy channel
microscopy_channel = MicroscopyChannel(
    name="gcamp_channel",
    description="GCaMP6f channel",
    excitation_wavelength_in_nm=920.0,
    emission_wavelength_in_nm=510.0,
    indicator=indicator,
)

## Define Volumetric Imaging Space and Create Example Data

Set up the volumetric imaging space parameters and create sample 4D data (time x height x width x depth). Note that volumetric imaging typically has:
- Different grid spacing in z compared to x,y
- Lower temporal resolution due to volume acquisition
- Larger data size due to the additional dimension

In [ ]:
# Create example volumetric data
frames = 100
height = 512
width = 512
depths = 10
data = np.random.rand(frames, height, width, depths)

# Define volumetric imaging space
illumination_pattern = LineScan(
    name="line_scanning",
    description="Line scanning two-photon microscopy",
    scan_direction="horizontal",
    line_rate_in_Hz=1000.0,
    dwell_time_in_s=1.0e-6,
)

volume_space = VolumetricImagingSpace(
    name="cortex_volume",
    description="Visual cortex volume",
    voxel_size_in_um=[1.0, 1.0, 2.0],  # Higher spacing in z
    dimensions_in_voxels=[height, width, depths],
    origin_coordinates=[-1.2, -0.6, -2.0],
    location="Visual cortex",
    reference_frame="bregma",
    orientation="RAS",  # Right-Anterior-Superior
    illumination_pattern=illumination_pattern,
)

# Create volumetric series
volume_series = VolumetricMicroscopySeries(
    name='volume_data',
    description='Volumetric imaging series',
    microscopy_channel=microscopy_channel,
    microscopy_rig=microscopy_rig,
    volumetric_imaging_space=volume_space,
    data=data,
    unit='a.u.',
    rate=5.0,  # Lower rate for volumetric imaging
    starting_time=0.0
)

nwbfile.add_acquisition(volume_series)

## 3D ROI Segmentation and Analysis

Create and analyze three-dimensional regions of interest (ROIs) in the volumetric data. This differs from planar ROI analysis by:
- Using VolumetricSegmentation instead of PlanarSegmentation
- Creating 3D ROI masks that span multiple z-planes
- Supporting both image-based and voxel-based ROI definitions

In [ ]:
# Create ophys processing module
ophys_module = nwbfile.create_processing_module(
    name='ophys',
    description='Optical physiology processing module'
)

# Create 3D summary images
mean_image = SummaryImage(
    name='mean',
    description='Mean intensity projection',
    data=np.mean(data, axis=0)
)

max_image = SummaryImage(
    name='max',
    description='Maximum intensity projection',
    data=np.max(data, axis=0)
)

# Create 3D segmentation
segmentation = VolumetricSegmentation(
    name='volume_rois',
    description='3D ROI segmentation',
    volumetric_imaging_space=volume_space,
    summary_images=[mean_image, max_image]
)

In [ ]:
# Add 3D ROIs using image masks
roi_mask = np.zeros((height, width, depths), dtype=bool)
roi_mask[256:266, 256:266, 4:6] = True  # 10x10x2 ROI
segmentation.add_roi(volume_mask=roi_mask)

# Create ROI responses
roi_region = segmentation.create_roi_table_region(
    description='All 3D ROIs',
    region=list(range(len(segmentation.id)))
)

# Extract responses (example calculation)
num_rois = len(segmentation.id)
responses = np.zeros((frames, num_rois))

for i, roi_mask in enumerate(segmentation.volume_mask[:]):
    roi_data = data[:, roi_mask]
    responses[:, i] = np.mean(roi_data, axis=1)

# Create response series
response_series = MicroscopyResponseSeries(
    name='volume_responses',
    description='Fluorescence responses from 3D ROIs',
    data=responses,
    rois=roi_region,
    unit='n.a.',
    rate=5.0,
    starting_time=0.0,
    microscopy_series=volume_series,    
)

# Create container for response series
response_container = MicroscopyResponseSeriesContainer(
    name='volume_responses',
    microscopy_response_series=[response_series]
)

# Add segmentation and responses to ophys module
ophys_module.add(segmentation)
ophys_module.add(response_container)

## Save and Load NWB File

Write the NWB file to disk and demonstrate how to read back the volumetric data and 3D ROIs.

In [ ]:
# Save file
with NWBHDF5IO('volumetric_imaging.nwb', 'w') as io:
    io.write(nwbfile)

# Read file and access data
with NWBHDF5IO('volumetric_imaging.nwb', 'r') as io:
    nwbfile = io.read()
    
    # Access volumetric data
    imaging = nwbfile.acquisition['volume_data']
    volume_data = imaging.data[:]
    
    # Access ROI data
    ophys = nwbfile.processing['ophys']
    rois = ophys['volume_rois']
    roi_masks = rois.volume_mask[:]
    
    # Access responses
    responses = ophys['volume_responses']
    roi_data = responses['volume_responses'].data[:]